In [20]:
import numpy as np
import h5py
import netCDF4
from glob import glob
import pandas as pd
import ipywidgets as widgets
import json

In [25]:
# Functions and variables
k = 1.38*10**(-23) # J/K
def Ne_convert(e: float,P: float,T: float) -> float:
    return e*P/(k*T)

def z(p):
    return - 7 * np.log(p/1013.25)

def format_time(hours_float):
    hours = int(hours_float)
    minutes_float = (hours_float - hours) * 60
    minutes = int(minutes_float)
    formatted_time = f'{hours:02}:{minutes:02}:00Z'
    return formatted_time

time_save = np.arange(0,24,0.5) 
time_bin = time_save[1]
H_save = np.arange(90,130,10)
h_bin = H_save[1]
bins = len(time_save) * len(H_save) 

In [26]:
data_folder_path = '../../DataSorted/*'
data_folders = glob(data_folder_path)
data_folders = sorted(data_folders)
print(len(data_folders), 'folders found.')

dst_index_path = '../../DST_index/global_dst.csv'
dst_index_file = pd.read_csv(dst_index_path)
print(len(dst_index_file), 'days founds.')

hp30_index_path = '../../hp30_index/global_hp30.json'
with open(hp30_index_path, 'r') as file:
    hp30_index_file = json.load(file)
    hp30_date = np.array(hp30_index_file['datetime'])
    hp30_index = np.array(hp30_index_file['Hp30'])
print(int(len(hp30_index_file['datetime'])/48), 'days founds.')

278 folders found.
1097 days founds.
1097 days founds.


In [27]:
# Array for all values we want
Ne_EXP_global = np.array([])
dNe_EXP_global = np.array([])
Ne_WACCM_global = np.array([])
mag_EXP_global = np.array([])
dmag_EXP_global = np.array([])
mag_WACCM_global = np.array([])
Date_Global = np.array([])
Geo_Event = np.array([])
Solar_Event = np.array([])
Svalbard_Data = np.array([])
Tromso_Data = np.array([])
Height_Global = np.array([])
Hours_Global = np.array([])
dst_global = np.array([])
ddst_global = np.array([])
hp30_global = np.array([])
dhp30_global = np.array([])

folder_index = 1

progress_bar = widgets.IntProgress(
    value=folder_index,
    min  = 0,
    max  = len(data_folders),
    description = 'Progress:',
    bar_style   = 'info',
    orientation = 'horizontal'
)
display(progress_bar)

# Iterating over folders
for folder in data_folders:
    # Glob path for each files
    EXP_path = folder + '/MAD*.hdf5'
    WACCM_path = folder + '/*.nc'

    WACCM_files = []

    if len(glob(WACCM_path)) > 1:
        for file in glob(WACCM_path):
            WACCM_files.append(netCDF4.Dataset(file))
    elif len(glob(WACCM_path)) == 1:
        WACCM_files = [netCDF4.Dataset(glob(WACCM_path)[0])]
    else:
        WACCM_files = []
        break

    # Iterating over all experiment file
    for file in glob(EXP_path): 
        EXP_file = h5py.File(file)
        
        data = EXP_file['Data']['Table Layout'][:] # Get data from the file
        metadata = EXP_file['Metadata']['Data Parameters'][:] # Get data parameters from the file
        parameters = [parameter[0] for parameter in metadata] # Get the name of each parameters

        data = np.array([np.array(tuple.tolist()) for tuple in data])
        dataframe = pd.DataFrame(data, columns=parameters)
    
        h_start = np.array(dataframe[b'GDALT']) # Start Altitude km
        h_end = np.array(dataframe[b'RANGE']) # END Altitude km
        Ne = np.array(dataframe[b'NE']) # Electron density m-3
        dNe = np.array(dataframe[b'DNE']) # Electron density error m-3
        Ti = np.array(dataframe[b"TI"]) # Ion temperature K
        Tr = np.array(dataframe[b'TR']) # Electron to ion temperature ratio
        hours = np.array(dataframe[b'HOUR']) # Hours
        minutes = np.array(dataframe[b'MIN']) # Minutes

        # Restrict experiment values to below 150 km
        h_mask = h_end < 150
        h_start = h_start[h_mask]
        h_end = h_end[h_mask]
        Ne = Ne[h_mask]
        dNe = dNe[h_mask]
        Ti = Ti[h_mask]
        Tr = Tr[h_mask]
        Te = Ti * Tr
        hours = hours[h_mask]
        minutes = minutes[h_mask]
        time = hours + minutes/60

        lat = WACCM_files[0]['instr_lat'][:] # Latitude of the instrument
        lon = WACCM_files[0]['instr_lon'][:] # Longitude of the instrument
        num = WACCM_files[0]['instr_num'][:] # Numerical identifier of the instrument
        date = WACCM_files[0]['obs_date'][:] # Observation date
        time_WACCM = WACCM_files[0]['obs_time'][:] # Observation time
        lev = WACCM_files[0]['lev'][:] # 88 levels
        e = WACCM_files[0]['e'][:] # Mixing ratio
        T = WACCM_files[0]['T'] [:]# Temperature

        for WACCM_file in WACCM_files[1:]:
            temp_lat = WACCM_file['instr_lat'][:] # Latitude of the instrument
            temp_lon = WACCM_file['instr_lon'][:] # Longitude of the instrument
            temp_num = WACCM_file['instr_num'][:] # Numerical identifier of the instrument
            temp_date = WACCM_file['obs_date'][:] # Observation date
            temp_time = WACCM_file['obs_time'][:] # Observation time
            temp_e = WACCM_file['e'][:] # Mixing ratio 
            temp_T = WACCM_file['T'][:] # Temperature

            # Concatenate all model values into their unique array
            lat = np.concatenate((lat, temp_lat))
            lon = np.concatenate((lon, temp_lon))
            num = np.concatenate((num, temp_num))
            date = np.concatenate((date, temp_date))
            time_WACCM = np.concatenate((time_WACCM, temp_time))
            e = np.concatenate((e, temp_e))
            T = np.concatenate((T, temp_T))
        
        # Date restriction
        EXP_date = int(dataframe[b'YEAR'][0]*10000 + dataframe[b'MONTH'][0]*100 + dataframe[b'DAY'][0])
        date_mask = date == EXP_date
        lat = lat[date_mask]
        lon = lon[date_mask]
        time_WACCM = time_WACCM[date_mask]
        e = e[date_mask]
        T = T[date_mask]

        # Coordinate restrictions for Tromsø and Svalbard
        mask_tromso = (lat > 69.5) & (lat < 69.7)
        mask_svalbard = (lat > 78.8) & (lat < 79.0)

        if 'uhf' in file: # Check if this is a Tromsø experiment
            mask_location = mask_tromso
            Tromso_Data = np.concatenate((Tromso_Data, np.repeat(True, bins))) # See description.md for explanation of this part
            Svalbard_Data = np.concatenate((Svalbard_Data, np.repeat(False, bins))) # See description.md for explanation of this part
        elif '42m' in file: # Check if this is a Svalbard experiment
            mask_location = mask_svalbard
            Tromso_Data = np.concatenate((Tromso_Data, np.repeat(False, bins))) # See description.md for explanation of this part
            Svalbard_Data = np.concatenate((Svalbard_Data, np.repeat(True, bins))) # See description.md for explanation of this part

        # Pressure restriction for model value, 0.01 approx 80 km
        lev_mask = lev < 0.01
        P = lev[lev_mask]
        H = z(P)
        NeWACCM = [] # Be careful this is not density but mixing ratio
        for array in e[mask_location]:
            NeWACCM.append(array[lev_mask])
        NeWACCM = np.array(NeWACCM)

        hours = time_WACCM[mask_location]/3600
        T = T[mask_location]

        # Split folder path to get specifif event 
        date = folder.split('/')[-1]

        if 'GEO' in date and 'SOL' in date: # Check if it's both
            date = date.replace('-GEO', '')
            date = date.replace('-SOL', '')
            Geo_Event = np.concatenate((Geo_Event, np.repeat(True, bins))) # See description.md for explanation of this part
            Solar_Event = np.concatenate((Solar_Event, np.repeat(True, bins))) # See description.md for explanation of this part
        elif 'GEO' in date and 'SOL' not in date: # Check if it's Geomagnetic event
            date = date.replace('-GEO', '')
            Geo_Event = np.concatenate((Geo_Event, np.repeat(True, bins))) # See description.md for explanation of this part
            Solar_Event = np.concatenate((Solar_Event, np.repeat(False, bins))) # See description.md for explanation of this part
        elif 'GEO' not in date and 'SOL' in date: # Check if it's Solar event
            date = date.replace('-SOL', '')
            Geo_Event = np.concatenate((Geo_Event, np.repeat(False, bins))) # See description.md for explanation of this part
            Solar_Event = np.concatenate((Solar_Event, np.repeat(True, bins))) # See description.md for explanation of this part
        else:
            Geo_Event = np.concatenate((Geo_Event, np.repeat(False, bins))) # See description.md for explanation of this part
            Solar_Event = np.concatenate((Solar_Event, np.repeat(False, bins))) # See description.md for explanation of this part

        # Create list for ease of use (append)
        Ne_EXP_list = []
        dNe_EXP_list = []
        Ne_WACCM_list = []
        dst_list = []
        ddst_list = []
        hp30_list = []
        dhp30_list = []

        # Mask based on the date for the DST index 
        dst_mask = np.array(dst_index_file['DATE'] == date) 
        # Shift the DST mask to get the next day after the current date
        dst_shifted = np.roll(dst_mask,1) 
        
        # Iterating over all altitude slot 
        for height_slot in H_save:
            h_start_mask = (h_start - height_slot <= h_bin) & (h_start - height_slot > 0) # Check if start altitude of the measurements if in the slot (Experiment)
            h_end_mask = (h_end - height_slot <= h_bin) & (h_end - height_slot > 0) # Check if end altitude of the measurements if in the slot (Experiment)
            h_mask_EXP = h_start_mask + h_end_mask # Combine both experiment mask
            h_mask_WACCM = (H - height_slot <= h_bin) & (H - height_slot > 0) # Check if model altitude is in the slot
            # Iterating over all time slot 
            for time_slot in time_save:
                t_mask_EXP = (time[h_mask_EXP] - time_slot <= time_bin) & (time[h_mask_EXP] - time_slot > 0) # Check if the start time of the experiment is in slot(Experiment)
                t_mask_WACCM = (hours - time_slot <= 0.5) & (hours - time_slot > 0) # Check if the model time is in the slot
                if sum(t_mask_EXP) == 0: # Check if there is no experiment measure in this bin (time slot x altitude slot)
                    # Add 0 for each value
                    Ne_EXP_list.append(0)
                    dNe_EXP_list.append(0)
                    Ne_WACCM_list.append(0)
                    dst_list.append(0)
                    ddst_list.append(0)
                    hp30_list.append(0)
                    dhp30_list.append(0)
                elif sum(t_mask_WACCM) == 0:# Check if there is no model estimation in this bin (time slot x altitude slot)
                    # Add 0 for each value
                    Ne_EXP_list.append(0)
                    dNe_EXP_list.append(0)
                    Ne_WACCM_list.append(0)
                    dst_list.append(0)
                    ddst_list.append(0)
                    hp30_list.append(0)
                    dhp30_list.append(0)
                else:
                    Ne2_EXP = np.mean(Ne[h_mask_EXP][t_mask_EXP]) # Get the mean value of the experiment measures in this bin
                    Ne_EXP_list.append(Ne2_EXP) # Add this mean to the list
                    dNe_EXP = np.sqrt(np.sum(dNe[h_mask_EXP][t_mask_EXP]**2))/len(dNe[h_mask_EXP][t_mask_EXP]) # Get the squared root of the sum of squared error (see description.md for further explanation)
                    dNe_EXP_list.append(dNe_EXP) # Add this value to the list

                    Ne2_WACCM_array = np.array([]) #
                    index = 0
                    for index in range(len(NeWACCM[t_mask_WACCM])): # Iterating over all mixing ratios arrays in the array
                        Ne_array = NeWACCM[t_mask_WACCM][index][h_mask_WACCM] # Get the current mixing ration array
                        T_array = T[t_mask_WACCM][index][lev_mask][h_mask_WACCM] # Get the current temperature array
                        # Convert mixing ratios to density and add them to the global array (see description.md for further explanation)
                        Ne2_WACCM_array = np.concatenate((Ne2_WACCM_array, Ne_convert(Ne_array,P[h_mask_WACCM]*100,T_array))) 
                    Ne2 =  np.mean(Ne2_WACCM_array) # Mean the densities
                    Ne_WACCM_list.append(Ne2) # Add this mean to the list

                    int_hour_str = str(int(time_slot) + 1) # As DST values are shifted we need to shift or time slot by one hour
                    next_int_hour_str = str(int(time_slot + time_bin) + 1) # Get next hour we will use
                    dst = dst_index_file[int_hour_str][dst_mask].values[0] # Get current DST value

                    # Further explanation of this part in description.md
                    if (time_bin == 1 or int_hour_str == next_int_hour_str) and int_hour_str != '24':
                        dst_list.append(dst) # Add the DST value
                        next_dst = dst_index_file[next_int_hour_str][dst_mask].values[0] # Get the next DST value
                        ddst_list.append((next_dst - dst)/time_bin) # Add the gradient
                    elif int_hour_str != next_int_hour_str and int_hour_str != '24': # Check is the two hour are the same and the first is not 24
                        next_dst = dst_index_file[next_int_hour_str][dst_mask].values[0] # Get next DST value
                        dst_list.append((dst + next_dst)/2) # Add the mean of the two DST value
                        ddst_list.append((next_dst - dst)/(2*time_bin)) # Add the gradient
                    else: # this happens only when HH:mm = 24:00
                        dst_list.append(dst) # add the DST value
                        next_dst = dst_index_file['1'][dst_shifted].values[0] # Get the next DST value
                        ddst_list.append((next_dst - dst)/time_bin) # Add the gradient

                    # Date mask for Hp30 index , for which time slot must be formatted
                    hp30_mask = hp30_date == f'{date}T{format_time(time_slot)}'
                    hp30_shifted = np.roll(hp30_mask, 1) # Shift the mask for the gradient
                    hp30 = hp30_index[hp30_mask][0] # Get current Hp30 value
                    next_hp30 = hp30_index[hp30_shifted][0] # Get next Hp30 value
                    dhp30 = (next_hp30 - hp30)/time_bin # Compute the gradient
                    hp30_list.append(hp30) # Add the value
                    dhp30_list.append(int(dhp30*100)/100) # Add the gradient (with 3 significant digits)
                    
            Hours_Global = np.concatenate((Hours_Global, time_save)) # See description.md for explanation of this part
            Height_Global = np.concatenate((Height_Global, np.repeat(height_slot, len(time_save)))) # See description.md for explanation of this part

        # Convert all list to array, and fix their types if needed
        Ne_EXP = np.array(Ne_EXP_list).astype(int)
        dNe_EXP = np.array(dNe_EXP_list).astype(int)
        Ne_WACCM = np.array(Ne_WACCM_list).astype(int)
        dst_array = np.array(dst_list)
        ddst_array = np.array(ddst_list)
        hp30_array = np.array(hp30_list)
        dhp30_array = np.array(dhp30_list)

        # Extract nan value that can occurs in the experiment
        nan_mask = np.isnan(Ne_EXP)
        Ne_EXP[nan_mask] = 0
        Ne_WACCM[nan_mask] = 0

        # Create empty magnitude array
        mag_EXP = np.zeros((len(H_save) * len(time_save)))
        dmag_EXP = np.zeros((len(H_save) * len(time_save)))
        mag_WACCM = np.zeros((len(H_save) * len(time_save)))
        
        # Take care of the 0 values in the array
        mask_zero = Ne_EXP != 0
        # Compute their magnitude
        mag_EXP[mask_zero] = np.log10(Ne_EXP[mask_zero])
        dmag_EXP[mask_zero] = np.log10(dNe_EXP[mask_zero])
        mag_WACCM[mask_zero] = np.log10(Ne_WACCM[mask_zero])

        # Correct the values with 3 significant digits
        mag_EXP = (mag_EXP*100).astype(int)/100
        dmag_EXP = (dmag_EXP*100).astype(int)/100
        mag_WACCM = (mag_WACCM*100).astype(int)/100

        # Concatenate all current arrays with their global arrays
        Ne_EXP_global = np.concatenate((Ne_EXP_global, Ne_EXP))
        dNe_EXP_global = np.concatenate((dNe_EXP_global, dNe_EXP))
        Ne_WACCM_global = np.concatenate((Ne_WACCM_global, Ne_WACCM))
        
        mag_EXP_global = np.concatenate((mag_EXP_global, mag_EXP))
        dmag_EXP_global = np.concatenate((dmag_EXP_global, dmag_EXP))
        mag_WACCM_global = np.concatenate((mag_WACCM_global, mag_WACCM))

        dst_global = np.concatenate((dst_global, dst_array))
        ddst_global = np.concatenate((ddst_global, ddst_array))
        hp30_global = np.concatenate((hp30_global, hp30_array))
        dhp30_global = np.concatenate((dhp30_global, dhp30_array))

        Date_Global = np.concatenate((Date_Global, np.repeat(date, bins)))
    
    folder_index += 1
    progress_bar.value = folder_index

IntProgress(value=1, bar_style='info', description='Progress:', max=278)

In [24]:
data_dict = {'Date' : Date_Global,
             'Hours' : Hours_Global,
             'Height' : Height_Global.astype(int),
             'Svalbard' : Svalbard_Data.astype(int),
             'Tromso' : Tromso_Data.astype(int),
             'Geomagnetic Event' : Geo_Event.astype(int),
             'Solar Proton Event' : Solar_Event.astype(int),
             'EXP Density' : Ne_EXP_global.astype(int),
             'EXP Density Error' : dNe_EXP_global.astype(int),
             'WACCM Density' : Ne_WACCM_global.astype(int),
             'EXP Magnitude' : mag_EXP_global,
             'EXP Magnitude Error' : dmag_EXP_global, 
             'WACCM Magnitude' : mag_WACCM_global,
             'DST Index' : dst_global,
             'DST Index Gradient' : ddst_global,
             'Hp30 Index' : hp30_global,
             'Hp30 Index Gradient' : dhp30_global,
            }

data_df = pd.DataFrame(data_dict)
file = f'global_data_{len(H_save)}_{len(time_save)}'
data_df.to_csv('../../Results/' + file + '.csv', index=False)